### Imort Dependencies

In [12]:
import os
import torch
from torchvision import transforms, models
from PIL import Image, ImageDraw
from IPython.display import display
import torch.nn as nn
from annoy import AnnoyIndex
import time

### Load Data and vectorise it, then add the vector to annoy index

In [20]:

path = "/Volumes/Seagate Bac/projects/image_similarity_data/"
images = [i for i in os.listdir(path) if not i.startswith(".")]

weights = models.ResNet18_Weights.IMAGENET1K_V1
model = models.resnet18(weights=weights)

# weights = models.ResNet50_Weights.IMAGENET1K_V2
# model = models.resnet50(weights=weights)

model.fc = nn.Identity()
model.eval()

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

# annoy_index = AnnoyIndex(512, 'angular')
annoy_index = AnnoyIndex(512, 'angular')
annoy_index.load("./temp_index.ann")

True

### Make inference

In [21]:
image_grid = Image.new('RGB',(1000,1000))

image = Image.open("path")
# display(image)
input_tensor = transform(image).unsqueeze(0)

if input_tensor.size()[1] == 3:
    output_tensor = model(input_tensor)

    start = time.time()
    nns = annoy_index.get_nns_by_vector(output_tensor[0],24,search_k=-1, include_distances=False)
    print("Time taken: ", time.time()-start, " seconds")

    image = image.resize((200,200))
    image_draw = ImageDraw.Draw(image)
    image_draw.rectangle([(0,0),(199,199)],outline='red',width=8)
    image_grid.paste(image,(0,0))

    for j in range(len(nns)):
        search_image = image = Image.open(os.path.join(path , images[nns[j]]) )
        search_image = search_image.resize((200,200))
        image_grid.paste(search_image,(200*((j+1)%5),200*((j+1)//5)))


Time taken:  0.004021883010864258


In [18]:
image_grid.save("temp3_1.png")